In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### important note : first i will do feature selcetion for buliding a model after that for next days i will complete notebook with data visualization

# Lets load data

In [ ]:
df_train = pd.read_csv('../input/loan-eligible-dataset/loan-train.csv')
df_test = pd.read_csv('../input/loan-eligible-dataset/loan-test.csv')

In [ ]:
df_train.head()

the Loan_ID column is not useful for us and building a model then we must drop it

In [ ]:
df_train.drop(columns='Loan_ID', inplace = True)
df_test.drop(columns='Loan_ID', inplace = True)

Lets get more data from our columns by df.info()

In [ ]:
df_train.info()

In [ ]:
df_train.describe().transpose()

In [ ]:
sns.set(style = 'whitegrid')
plt.figure(figsize = (16,8))
plt.subplot(2,2,1)
sns.countplot(x = 'Gender' , data = df_train)
plt.subplot(2,2,2)
sns.countplot(x = 'Married', data= df_train)
plt.subplot(2,2,3)
sns.countplot(x = 'Gender', hue= 'Loan_Status', data = df_train)
plt.subplot(2,2,4)
sns.countplot(x = 'Married',hue = 'Loan_Status' , data= df_train)
plt.show()

In [ ]:
sns.pairplot(df_train)
plt.show()

Its look like ApplicantIncome and LoanAmount have a linear relationship (or poly?) , and this relationship is make sense cause more income more loan amount can affort .

## lets check it more

In [ ]:
plt.figure(figsize = (16,8))
sns.scatterplot(x = 'LoanAmount' , y = 'ApplicantIncome' , hue = 'Loan_Status', data = df_train)
plt.show()

And it is look like there is no obvious relationship between these 2 features and Loan_Status

## Lets check category variables more

In [ ]:
sns.set(style = 'whitegrid')
plt.figure(figsize = (16,8))
plt.subplot(2,2,1)
sns.countplot(x = 'Education' , data = df_train)
plt.subplot(2,2,2)
sns.countplot(x = 'Property_Area', data= df_train)
plt.subplot(2,2,3)
sns.countplot(x = 'Education', hue= 'Loan_Status', data = df_train)
plt.subplot(2,2,4)
sns.countplot(x = 'Property_Area',hue = 'Loan_Status' , data= df_train)
plt.show()

In [ ]:
graduate = df_train['Education'].value_counts()['Graduate']
not_graduate = df_train['Education'].value_counts()['Not Graduate']
graduate_yes = df_train[df_train['Loan_Status'] == 'Y']['Education'].value_counts()['Graduate']
not_graduate_yes = df_train[df_train['Loan_Status'] == 'Y']['Education'].value_counts()['Not Graduate']

In [ ]:
print(graduate_yes / graduate)
print(not_graduate_yes / not_graduate)

oh its look like if you are graduate you have more possiblity to take Loan

what percentage of people are self employed? and what percentage of people can take a loan?

In [ ]:
plt.figure(figsize = (12,4))
plt.subplot(1,2,1)
plt.pie(df_train['Self_Employed'].value_counts(),explode = [0,0.2] , autopct='%.1f%%' , shadow = True , labels = ['No', 'Yes'])
plt.title('percent of values in Self employed column')
plt.subplot(1,2,2)
plt.pie(df_train['Loan_Status'].value_counts(), explode = [0,0.2], autopct = '%.1f%%' , labels = ['N', 'Y'])
plt.title('percent of acceptance')
plt.show()

In [ ]:
plt.figure(figsize = (10,5))
sns.countplot('Self_Employed' , hue = 'Loan_Status' , data = df_train)
plt.show()

number of who are not self employed is high so the number of them who can take loan is higher than the rest

In [ ]:
sns.boxplot(x = 'Gender', y='ApplicantIncome' , data = df_train)
plt.show()

it is showing us that men have higher median income than women and there are someones that have income more than 20 thousends dollar at year even its rising up to 80 thousends dollar but all of the women have 20 thousends dollar and lower .

In [ ]:
plt.bar('mean of Applicant Income',df_train['ApplicantIncome'].mean())
plt.bar('mean of Loan Amount', df_train['LoanAmount'].mean())
plt.show()

for the columns that are object type we need to take care of them by turn them into numerical columns.\
Now we need to know what value they have

In [ ]:
print('Unique values in Gender :',df_train['Gender'].unique())
print('Unique values in Married :',df_train['Married'].unique())
print('Unique values in Dependents :',df_train['Dependents'].unique())
print('Unique values in Education :',df_train['Education'].unique())
print('Unique values in Self Employed :',df_train['Self_Employed'].unique())
print('Unique values in Property Area :',df_train['Property_Area'].unique())
print('Unique values in Loan Status :',df_train['Loan_Status'].unique())

now when we know their value , we can turn them to numeric.\
it could be done by 3 ways ( at least the ways i know :) ).\
1.using replace method like what we are going to do\
2.using map method then we should build a dictionary\
3.using Lable encoder from sklearn library

In [ ]:
df_train['Gender'] = df_train['Gender'].replace(['Male','Female'],[1,0])

In [ ]:
df_train['Married'] = df_train['Married'].replace(['Yes','No'],[1,0])
df_train['Dependents'] = df_train['Dependents'].replace(['0','1','2'],[0,1,2])
df_train['Dependents'] = df_train['Dependents'].replace('3+' , 3)
df_train['Education'] = df_train['Education'].replace(['Graduate' , 'Not Graduate'],[1,0])
df_train['Self_Employed'] = df_train['Self_Employed'].replace(['Yes','No'],[1,0])
df_train['Property_Area'] = df_train['Property_Area'].replace(['Urban' ,'Rural' ,'Semiurban'],[0,1,2])
df_train['Loan_Status'] = df_train['Loan_Status'].replace(['Y','N'],[1,0])

In [ ]:
df_train.describe()

Lets find out how many missing value we have

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

## Lets check if we drop the missing values then data is going to skewed or not

In [ ]:
df_temp = df_train.dropna()

In [ ]:
df_temp.describe()

By comparing this two describe table , we realized that the mean for every column change near 1 percent then its a good technique 

the Property Area column has 3 diffrent value , since we turn them to numric maybe there is no order between them so its better to have one column for every value.\
it could be done by 2 ways.\
first using pd.get_dummies method\
second using One Hot Encoder from sklearn library\
i think the first one is simple than the second 

In [ ]:
df_temp = pd.get_dummies(df_temp , columns = ['Property_Area'])

In [ ]:
df_temp

# Lets plot the Correlation Matrix
to find the high correlated columns with Loan Status

In [ ]:
corr = df_temp.corr()
plt.figure(figsize = (16,8))
sns.heatmap(corr , annot = True )
plt.show()

droping the nan value from test set

In [ ]:
df_test.dropna(inplace = True)

In [ ]:
df_test['Property_Area'] = df_test['Property_Area'].replace(['Urban' ,'Rural' ,'Semiurban'],[0,1,2])

In [ ]:
df_test = pd.get_dummies(df_test , columns = ['Property_Area'])

In [ ]:
df_test['Married'] = df_test['Married'].replace(['Yes', 'No'], [1,0])

In [ ]:
df_tr_x = df_temp[['Married' , 'Credit_History' , 'Property_Area_1', 'Property_Area_2']].values
df_tr_y = df_temp['Loan_Status'].values
df_ts = df_test[['Married', 'Credit_History', 'Property_Area_1', 'Property_Area_2']].values

# Lets build models

In [ ]:
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import classification_report,plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df_tr_x , df_tr_y , test_size = 0.25)

In [ ]:
cls_lr = LogisticRegression()
cls_lr.fit(x_train, y_train)
y_pred = cls_lr.predict(x_test)
print(classification_report(y_test , y_pred))
plot_confusion_matrix(cls_lr , x_test, y_test)
plt.show()

Accuracy is not bad but the f1 score and recall is low.\
let try with KFold Cross Validation to find the mean Accuracy

In [ ]:
cross_val = cross_val_score(cls_lr , x_train, y_train , cv=10)
print(cross_val.mean())

well the accuracy is even lower than the first try.\
lets try Random Forest

In [ ]:
cls_rf = RandomForestClassifier()
cls_rf.fit(x_train, y_train)
y_pred = cls_rf.predict(x_test)
print(classification_report(y_test , y_pred))
plot_confusion_matrix(cls_rf, x_test, y_test)
plt.show()

Surprisingly the accuracy of Random Forest and Logistic Regressin is the same.\
not only accuracy but also f1-score , recall and etc in the classification report

In [ ]:
cross_val = cross_val_score(cls_rf , x_train, y_train , cv=10)
print(cross_val.mean())

lets check the K Nearest Neighbors 

In [ ]:
results = []
for i in range(1,25): 
    cls_knn_2 = KNeighborsClassifier(n_neighbors=i)
    cls_knn_2.fit(x_train, y_train)
    score = cls_knn_2.score(x_test ,y_test)
    results.append([score])

In [ ]:
plt.plot(list(range(1,25)) , results)
plt.show()

above plot show us that with k is higher than 11 we have highest accuracy except when k equals to 20

In [ ]:
cls_knn = KNeighborsClassifier(n_neighbors=11)
cls_knn.fit(x_train, y_train)
predicted = cls_knn.predict(df_ts)

In [ ]:
predicted

# to be continued .....

## if you like this notebook please Upvote , thank you ! 
## i will be happy if you share your coments about this notebook :)